# Add _df to all dfs

In [15]:
import pandas as pd
from datetime import timedelta

In [50]:
# Read in data
# products = pd.read_csv("../Resources/products.csv")
# accounts = pd.read_csv("../Resources/accounts.csv")
ast_df = pd.read_csv("../Resources/account_state_transitions.csv")
# gpa = pd.read_csv("../Resources/group_product_associations.csv")
# groups = pd.read_csv("../Resources/groups.csv")
# payments = pd.read_csv("../Resources/payments.csv")

In [51]:
ast_df.head()

,id,created_when,started_when,account_id,from_state,to_state,previous_state_transition_id
0,3,2020-03-12 20:03:35.724574,2019-11-06 05:49:39.571392,2,CREATED,ENABLED,NaN
1,4,2020-03-12 20:03:35.724574,2019-11-25 22:20:59.150339,2,ENABLED,DISABLED,3.0
2,5,2020-03-12 20:03:36.042456,2019-11-26 10:22:36.571392,2,DISABLED,ENABLED,4.0
3,6,2020-03-12 20:03:36.042456,2019-11-29 11:03:32.150339,2,ENABLED,DISABLED,5.0
4,7,2020-03-12 20:03:36.098448,2019-12-03 10:08:03.571392,2,DISABLED,ENABLED,6.0


## 2) Devise a method of generating a table of all Accounts with their State as of any arbitrary timestamp in the past and the total time they had been in that state as of the timestamp.

In [52]:
# State as of: date (Using 'started_when' for now)

# SELECT 'to_state' WHERE 'started_when' == timestamp

In [54]:
# Variable to take timestamp input.
timestamp_str = input("Enter date with YYYY-MM-DD format.")

Enter date with YYYY-MM-DD format.2020-03-01


In [55]:
# Convert variable from string to datetime object
timestamp_dt = pd.to_datetime(timestamp_str)

In [56]:
# Change data type from string to datetime object
ast_df['started_when'] = pd.to_datetime(ast_df['started_when'])

##  timedelta(days=1) = timedelta(hours=24) -> doesn't change at midnight/partial days

In [57]:
# Create limited dataframe based on indicated timestamp 
df_limited = ast_df.loc[(ast_df['started_when'] >= timestamp_dt) & (ast_df['started_when'] < timestamp_dt + timedelta(days=1))]
df_limited.head()

,id,created_when,started_when,account_id,from_state,to_state,previous_state_transition_id
894,4689,2020-03-12 20:05:14.950510,2020-03-01 06:59:59.571392,34,DISABLED,UNLOCKED,4688.0
1009,6862,2020-03-12 20:05:33.177894,2020-03-01 08:29:39.571392,38,DISABLED,ENABLED,6861.0
1045,7272,2020-03-12 20:05:37.621790,2020-03-01 22:46:39.571392,39,DISABLED,ENABLED,7271.0
1112,7963,2020-03-12 20:05:46.723718,2020-03-01 20:41:24.340623,42,ENABLED,DISABLED,7962.0
1407,9576,2020-03-12 20:06:43.954110,2020-03-01 16:07:29.571392,60,DISABLED,ENABLED,9575.0


## Try timestamp input with time HH:MM:SS

## Need to select "to_state" based on timestamp, then calculate timedelta based on Peter's answer

In [64]:
# Create dataframe to show current state of each account based on indicated timestamp
df_limited2 = df_limited[['started_when','account_id','to_state']]

# Sort by account_id
df_limited2 = df_limited2.sort_values('account_id')

# Reset index
df_limited2 = df_limited2.reset_index(drop=True)
df_limited2.head()

,started_when,account_id,to_state
0,2020-03-01 06:59:59.571392,34,UNLOCKED
1,2020-03-01 08:29:39.571392,38,ENABLED
2,2020-03-01 22:46:39.571392,39,ENABLED
3,2020-03-01 20:41:24.340623,42,DISABLED
4,2020-03-01 22:57:43.571392,55,ENABLED


In [81]:
df_test = pd.DataFrame([['2020-03-01 18:29:39.571392', 38, 'DISABLED']], columns=['started_when','account_id',"to_state"])
df_limited2 = df_limited2.append(df_test)

In [89]:
df_limited2.loc[df_limited2.account_id==38]

,started_when,account_id,to_state
1,2020-03-01 08:29:39.571392,38,ENABLED
0,2020-03-01 18:29:39.571392,38,DISABLED


In [92]:
# Check for accounts occuring more than once

# s = df2.account_id.value_counts()
# a = pd.DataFrame(s).reset_index()
# a.loc[a.account_id > 1]

account_ids = []

for i in range(0,len(df_limited2['account_id'])):
    account_id = df_limited2['account_id'][i]
    if account_id in account_ids:
        pass
    else:
        account_ids.append(account_id)

0    34
0    38
Name: account_id, dtype: int64
<class 'pandas.core.series.Series'>
38
<class 'numpy.int64'>
39
<class 'numpy.int64'>
42
<class 'numpy.int64'>
55
<class 'numpy.int64'>
60
<class 'numpy.int64'>
68
<class 'numpy.int64'>
73
<class 'numpy.int64'>
76
<class 'numpy.int64'>
89
<class 'numpy.int64'>
96
<class 'numpy.int64'>
110
<class 'numpy.int64'>
114
<class 'numpy.int64'>
115
<class 'numpy.int64'>
128
<class 'numpy.int64'>
131
<class 'numpy.int64'>
138
<class 'numpy.int64'>
154
<class 'numpy.int64'>
159
<class 'numpy.int64'>
163
<class 'numpy.int64'>
164
<class 'numpy.int64'>
168
<class 'numpy.int64'>
168
<class 'numpy.int64'>
194
<class 'numpy.int64'>
208
<class 'numpy.int64'>
209
<class 'numpy.int64'>
215
<class 'numpy.int64'>
221
<class 'numpy.int64'>
234
<class 'numpy.int64'>
239
<class 'numpy.int64'>
251
<class 'numpy.int64'>
253
<class 'numpy.int64'>
259
<class 'numpy.int64'>
272
<class 'numpy.int64'>
273
<class 'numpy.int64'>
277
<class 'numpy.int64'>
283
<class 'numpy

KeyError: 86